# Dealing with Grouping Stock Keeping Unit in Pandas Python

In [234]:
import pandas as pd
import numpy as np
import re

In previous part, we have discussed about how to cleanse data in pandas python. In this session, data has been clean in regular pattern.

In [214]:
data_main = pd.read_csv("sales data final.csv")
data_main.set_axis(['name','qty','std_code'],axis='columns',inplace=True)
data_main

,name,qty,std_code
0,bundling sk 15 ml - engrave edition-ib,3,skbd-ib15
1,bundling sk 15 ml - engrave edition-ib,17,skbd-ib15
2,bundling sk 15 ml - engrave edition-ib,10,skbd-ib15
3,bundling sk 15 ml - engrave edition-ib,13,skbd-ib15
4,bundling sk 15 ml - engrave edition-ib,1,skbd-ib15
...,...,...,...
4901,voke stock bulanan. 4 refillable cartridge - l...,19,sb-rm
4902,voke stock bulanan. 4 refillable cartridge - l...,11,sb-rm
4903,voke stock bulanan. 4 refillable cartridge - l...,8,sb-rm
4904,voke stock bulanan. 4 refillable cartridge - l...,6,sb-sf


### 1. Creating Data Model and Grouping Classification

In [241]:
data_model = pd.DataFrame()
data_model['item_code'] = data_main['std_code']
data_model['group'] = [bool(re.search('.*-',value)) for value in data_model.item_code]
data_model['qty'] = data_main['qty']

group = data_model[data_model.group == True]
single = data_model[data_model.group == False]

In [242]:
summary_main = data_model[['item_code','qty']].groupby('item_code').sum()
summary_group = data_model[data_model.group == True][['item_code','qty']].groupby('item_code').sum()
summary_single = data_model[data_model.group == False][['item_code','qty']].groupby('item_code').sum()

grouping code is located preceeding the hypen while the single sku product (as variant) is located after hypen.
Therefore, the grouping code should be extracted and translated into spesific products.

In [243]:
extract = summary_group.index.str.extract(r'(.*)-(.*)')
summary_group['group_name']=extract[0].values
summary_group['varian_name']=extract[1].values
summary_group.head()

,qty,group_name,varian_name
item_code,,,
sb-ib,19,sb,ib
sb-ik,30,sb,ik
sb-pp,14,sb,pp
sb-rm,38,sb,rm
sb-sf,18,sb,sf


### 2. Translating Grouping Code to Interpret Single Items

Summary Group has 3 distinct grouping code, consisting of sb, sb2, and skbd which each grouping code has different group with different qty. Therefore, dict item_qty/group should be made first in dictionary form.

In [244]:
ct_per_item = {'skbd':2,'sb':4,'sb2':8}
stick_per_item = {'skbd':1,'sb':0,'sb2':4}

summary_group['ct_per_item'] = [ct_per_item[value] for value in summary_group.group_name]
summary_group['stick_per_item'] = [stick_per_item[value] for value in summary_group.group_name]

summary_group['ct'] = summary_group.qty*summary_group.ct_per_item
summary_group['stick'] = summary_group.qty*summary_group.stick_per_item

summary_group.head()

,qty,group_name,varian_name,ct_per_item,stick_per_item,ct,stick
item_code,,,,,,,
sb-ib,19,sb,ib,4,0,76,0
sb-ik,30,sb,ik,4,0,120,0
sb-pp,14,sb,pp,4,0,56,0
sb-rm,38,sb,rm,4,0,152,0
sb-sf,18,sb,sf,4,0,72,0


### 3. Melting Table

Melting/unpivot table was required to create basic raw data so that easier to analyze.

In [223]:
temp_a = pd.melt(summary_group,id_vars=['group_name'],value_vars=['ct','stick'])
temp_b = summary_group[['varian_name','qty']]
temp_b.reset_index(inplace=True)
temp_b.set_axis(['item','variable','value'],axis='columns',inplace=True)

target_column = ['variable','value']
summary_single_2 = pd.concat([temp_a[target_column],temp_b[target_column]])
summary_single_2

,variable,value
0,ct,76
1,ct,120
2,ct,56
3,ct,152
4,ct,72
5,ct,9576
6,ct,18704
7,ct,9432
8,ct,9552
9,ct,9864


In [237]:
final_a = summary_single.reset_index()
final_a.set_axis(['variable','value'],axis='columns',inplace=True)
summary_final = pd.concat([final_a,summary_single_2])
display(summary_final.head())

,variable,value
0,bp,3544
1,ct,14025
2,ib,190
3,jk,228
4,lyd,27


In [233]:
summary_final.groupby('variable').sum()

,value
variable,
bp,3544
ct,109951
ib,1406
ib15,3286
ik,2368
ik15,6252
jk,228
lyd,27
mg,608
